# Preprocessing and Pipelines

In this module, you will learn how to do some simple preprocessing with your data. Additionally, we will look at how to prevent data leakage with scikit-learn's awesome pipelines.

<b>Functions and attributes in this lecture: </b>
- `pandas:` - Pandas package with alias `pd`
  - `.mean()` - Get the mean value of a dataframe
  - `.replace()` - Replaces values in a series with new values
  - `.drop()` - Drop certain colunns or rows
  - `.dropna()` - Drop the rows with missing values
  - `.fillna()` - Fill in the missing values with a spesific value
- `sklearn.preprocessing` - Submodule for preprocessing data
  - `StandardScaler()` - Scale the data
    - `.fit()` - Training the scaler on the data
    - `.transform()` - Tranforms data by scaling it
    - `.mean_` - Get the mean for the scaling
    - `.var_` - Get the variance for the scaling
- `sklearn.pipelines` - Submodule for assembeling pipelines
  - `Pipeline()` - Basic constructor for setting up a pipeline
    - `.fit()` - Training the pipeline on the data
    - `.predict()` - Predict values on new data
    - `.score()` - Get the score determined by the last model in the pipeline
    - `.named_steps` - Get the components of the pipeline

In [9]:
# Non-sklearn packages
import numpy as np
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Sklearn packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Importing the Titanic Dataset

In this section we will import the famous Titanic dataset and clean some of the missing values in it!

In [10]:
# The titanic dataset is inside the seaborn package
from seaborn import load_dataset

# Load the Titanic data set
titanic = load_dataset("titanic")
titanic.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


In [11]:
# Checking summary data of the dataset
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [12]:
# Information about the columns
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [13]:
# Remove the "deck" feature
titanic.drop(columns=["deck"] , inplace=True)
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True


In [14]:
# Fill in the mean age for those with missing age
mean_age = round(titanic["age"].mean(),1)
titanic["age"].fillna(mean_age, inplace=True)

In [15]:
# Check that the value has been filled in
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 79.4+ KB


In [16]:
# Drop the remining two rows
titanic.dropna(inplace=True)

In [17]:
# Check that we have no more missing values
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     889 non-null    int64   
 1   pclass       889 non-null    int64   
 2   sex          889 non-null    object  
 3   age          889 non-null    float64 
 4   sibsp        889 non-null    int64   
 5   parch        889 non-null    int64   
 6   fare         889 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        889 non-null    category
 9   who          889 non-null    object  
 10  adult_male   889 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        889 non-null    object  
 13  alone        889 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 86.1+ KB


### Choosing Relevant Features

Not all the features you are presented with are nessesarily useful for predicting the survived feature. We will now exclude some of the features to only consider those we believe will affect the survived column significantly.

In [18]:
# Checking our dataset
titanic.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True
5,0,3,male,29.7,0,0,8.4583,Q,Third,man,True,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,Cherbourg,yes,False


In [19]:
# Removing duplicate information
titanic.drop(columns=["embarked","class","who","adult_male","alive"],inplace=True)

In [20]:
# Encode the sex as 0 for female and 1 for male
titanic["ismale"] = titanic["sex"].replace({"female":0, "male":1})
titanic.drop(columns=["sex"],inplace=True)
titanic.head(10)


,survived,pclass,age,sibsp,parch,fare,embark_town,alone,ismale
0,0,3,22.0,1,0,7.2500,Southampton,False,1
1,1,1,38.0,1,0,71.2833,Cherbourg,False,0
2,1,3,26.0,0,0,7.9250,Southampton,True,0
3,1,1,35.0,1,0,53.1000,Southampton,False,0
4,0,3,35.0,0,0,8.0500,Southampton,True,1
5,0,3,29.7,0,0,8.4583,Queenstown,True,1
6,0,1,54.0,0,0,51.8625,Southampton,True,1
7,0,3,2.0,3,1,21.0750,Southampton,False,1
8,1,3,27.0,0,2,11.1333,Southampton,False,0
9,1,2,14.0,1,0,30.0708,Cherbourg,False,0


In [21]:
# Can look at the correlation matrix (embark town is not present!)
titanic.corr(numeric_only=True)

,survived,pclass,age,sibsp,parch,fare,alone,ismale
survived,1.000000,-0.335549,-0.074675,-0.034040,0.083151,0.255290,-0.206207,-0.541585
pclass,-0.335549,1.000000,-0.327949,0.081656,0.016824,-0.548193,0.138553,0.127741
age,-0.074675,-0.327949,1.000000,-0.231875,-0.178235,0.088602,0.177716,0.089435
sibsp,-0.034040,0.081656,-0.231875,1.000000,0.414542,0.160887,-0.584186,-0.116348
parch,0.083151,0.016824,-0.178235,0.414542,1.000000,0.217532,-0.583112,-0.247508
fare,0.255290,-0.548193,0.088602,0.160887,0.217532,1.000000,-0.274079,-0.179958
alone,-0.206207,0.138553,0.177716,-0.584186,-0.583112,-0.274079,1.000000,0.306985
ismale,-0.541585,0.127741,0.089435,-0.116348,-0.247508,-0.179958,0.306985,1.000000


In [22]:
# Drop the low-correlation columns and the embark town column
titanic.drop(columns = ["age","sibsp","parch","embark_town",], inplace=True)

In [23]:
# Our dataset 
titanic.head()

,survived,pclass,fare,alone,ismale
0,0,3,7.2500,False,1
1,1,1,71.2833,False,0
2,1,3,7.9250,True,0
3,1,1,53.1000,False,0
4,0,3,8.0500,True,1


## Standardizing the Values

It is useful to standardize the values before passing them into machine learning models. While this is not important for all machine learning models, it is important for many of them.

In [24]:
# Dividing into traning sets and testing sets
y = titanic["survived"]
X = titanic.drop(columns=["survived"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [25]:
# Importing and initializing a standard scaler estimator
from sklearn.preprocessing import StandardScaler

In [26]:
# Fitting the estimator on the training set (y_train is ignored)
scaler = StandardScaler()
scaler.fit(X_train.values)

StandardScaler()

In [27]:
# Getting the mean and variance of the training set
print(scaler.mean_)
print(scaler.var_)

[ 2.32773109 32.76836067  0.59327731  0.6605042 ]
[6.87550314e-01 2.64921948e+03 2.41299343e-01 2.24238401e-01]


In [28]:
# Scaling the training set and testing set in the same way
x_train_scaled = scaler.transform(X_train.values)
x_test_scaled = scaler.transform(X_test.values)

In [29]:
# Checking the output
print(x_train_scaled)
print(x_test_scaled)

[[-1.60124536  0.37097199  0.82798092  0.71693438]
 [-0.39524411 -0.38407115  0.82798092 -1.39482779]
 [-1.60124536  0.95374775 -1.2077573   0.71693438]
 ...
 [-1.60124536 -0.13287517  0.82798092 -1.39482779]
 [ 0.81075714  0.03121472 -1.2077573  -1.39482779]
 [ 0.81075714 -0.46850387  0.82798092  0.71693438]]
[[ 0.81075714 -0.48404673  0.82798092  0.71693438]
 [-1.60124536  1.69478677 -1.2077573  -1.39482779]
 [ 0.81075714 -0.41823242 -1.2077573  -1.39482779]
 ...
 [ 0.81075714 -0.33080382 -1.2077573  -1.39482779]
 [ 0.81075714 -0.48607119  0.82798092 -1.39482779]
 [-1.60124536 -0.04026691  0.82798092  0.71693438]]


In [30]:
# Training a logistic regression model
log_reg = LogisticRegression()
log_reg.fit(x_train_scaled,y_train)

LogisticRegression()

In [31]:
# Get the predictions and accuracy score
y_pred = log_reg.predict(x_test_scaled)
accuracy_score(y_test, y_pred)

0.7993197278911565

In [32]:
# This is not so good, since the data is unbalanced!
1-titanic["survived"].mean()

0.6175478065241844

## Creating a Pipeline for Our Data

We will now put our scaling and logistic regression into a pipeline so that it is more managable.

In [33]:
# Importing the Pipeline object
from sklearn.pipeline import Pipeline

In [42]:
# Creting a pipeline
pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("log_reg", LogisticRegression())
    ],
    verbose=True
)

In [43]:
# Fitting the pipeline
pipeline.fit(X_train,y_train)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ........... (step 2 of 2) Processing log_reg, total=   0.0s


Pipeline(steps=[('scaler', StandardScaler()),
                ('log_reg', LogisticRegression())],
         verbose=True)

In [50]:
# Have all the information in the named_steps attribute
pipeline.named_steps["scaler"].var_

array([6.87550314e-01, 2.64921948e+03, 2.41299343e-01, 2.24238401e-01])

In [45]:
# Can now use predict and X_test gets automatically scaled
pipeline.predict(X_test)

array([0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0])

In [119]:
# Can use score to get the accuracy score for logistic regression
print(sum(y_pred==y_test)/len(y_test))
pipeline.score(X_test,y_test)

0.7993197278911565


0.7993197278911565